In [84]:
import spacy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import os
import gensim
from gensim.corpora import Dictionary
from gensim.similarities import MatrixSimilarity
from gensim.models import ldamodel, doc2vec, LsiModel 
import nltk
# nltk.download('punkt')
import string
import csv
import math
import statistics
import datetime
from nltk.corpus import stopwords
from nltk.util import ngrams
# nltk.download('stopwords')
from collections import OrderedDict, Counter, namedtuple
import networkx as nx
import matplotlib.pyplot as plt
from networkx.readwrite import json_graph
from bokeh.plotting import figure, show, output_file
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.layouts import row, column
import random
import codecs, difflib, Levenshtein, distance
import rpy2
from datasketch import MinHash
import warnings
warnings.filterwarnings('ignore')
nlp = spacy.load('en_core_web_lg')
# nlp_all = spacy.load('xx_ent_wiki_sm')

In [93]:
scribe_unordered = pd.read_csv('ocr_test_magazine_arab_scribe_unordered.csv')
scribe_ordered = pd.read_csv('ocr_test_magazine_arab_scribe_ordered.csv')
scribe_htrc = pd.read_csv('vol_page_freq.csv')

In [94]:
scribe_ordered['contains_image'].fillna(value=False, inplace=True)

for index, row in scribe_ordered.iterrows():
    if math.isnan(row['page_number']):
        pgn = row['base_file_name'].split('imagefile')[0][-3:]
        pgn = pgn.split('_')[0]
        scribe_ordered.loc[index, 'page_number'] = int(pgn)

groupby_df = scribe_ordered.groupby('page_number')['google_vision_text'].apply(' '.join).reset_index()
scribe_ordered = scribe_ordered.drop_duplicates(subset=['page_number'], keep='first')
scribe_ordered = scribe_ordered.drop(columns='google_vision_text')
final_df = pd.merge(scribe_ordered, groupby_df, on='page_number', how='outer')
scribe_ordered = final_df.drop(columns='id')
scribe_ordered

,base_file_name,contains_image,page_number,google_vision_text
0,image_lucida_app/media/The_Scribe_And_The_Arab...,False,0.0,The Scribe\nTHE ARAB REVIEW\n4th Year - Vol. V...
1,image_lucida_app/media/The_Scribe_And_The_Arab...,False,1.0,ARAB OBSERVER\nThe Non-Aligned Weekly\n
2,image_lucida_app/media/The_Scribe_And_The_Arab...,False,2.0,CHOU EN-LAI IN THE U.A.R.\nTHE PROBLEM OF TRAN...
3,image_lucida_app/media/The_Scribe_And_The_Arab...,True,3.0,The\nScribe\nHE ARAB REVIEW\nCONTENTS\nPage\n...
4,image_lucida_app/media/The_Scribe_And_The_Arab...,False,4.0,Under the probing lens of\nof Truth PILGRIMAG...
5,image_lucida_app/media/The_Scribe_And_The_Arab...,False,5.0,The Scribe\nAll the carefully prepared commun...
6,image_lucida_app/media/The_Scribe_And_The_Arab...,False,6.0,January 1964\nwere forced to accept it. The e...
7,image_lucida_app/media/The_Scribe_And_The_Arab...,False,7.0,Mr. Chou-en-Lai's Visit to Cairo\nRE-AFFIRMAT...
8,image_lucida_app/media/The_Scribe_And_The_Arab...,False,8.0,The political scene\nlems. The survey include...
9,image_lucida_app/media/The_Scribe_And_The_Arab...,True,9.0,"The Scribe\nThe Chinese Premier, Chou En Lai,..."


In [89]:
#Process text for ordered and ocr text
def custom_tokenize(text):
    if not text:
#       print('The text to be tokenized is a None type. Defaulting to blank string.')
        text = ''
    return nltk.word_tokenize(text)

def process_text(page_number, df, punc):
    final_df = pd.DataFrame(columns=['page', 'lowercase', 'counts'])
    final_doc = []
    final_text = []
    for index, row in df.iterrows():
        page = []
        raw_text = row['google_vision_text']
        tokens = custom_tokenize(raw_text)
        for t in tokens:
            final_text.append(t.lower())
            page.append(t.lower())

        counts = Counter(page)
#         print(counts)
        df = pd.DataFrame.from_dict(counts, orient='index').reset_index()
#         print(df)
        df = df.rename(columns={'index':'lowercase', 0:'counts'})
        df['page'] = int(page_number)
    return df

In [91]:
scribe_od = pd.DataFrame()

for i in range(0, len(scribe_ordered.index)):
    df = process_text(scribe_ordered['page_number'].iloc[[i]], scribe_ordered.iloc[[i]], True)
    if i == 0:
        scribe_od = df
    else:
        frames = [scribe_od, df]
        scribe_od = pd.concat(frames)

scribe_od.to_csv('scribe_')

,lowercase,counts,page
0,the,4,0
1,scribe,1,0
2,arab,2,0
3,review,1,0
4,4th,1,0
5,year,2,0
6,-,1,0
7,vol,1,0
8,.,2,0
9,vii,1,0


In [76]:
scribe_htrc[0:1]['count']

0    1
Name: count, dtype: int64

In [83]:
final_df = scribe_htrc[0:0]
for i, row in scribe_htrc.iterrows():
    selected_row = scribe_htrc.iloc[[i]].copy()
    token_count = selected_row['count'].values.astype(np.int64)[0]
#     print(type(token_count))
    for i in range(0, token_count):
        final_df = final_df.append(selected_row, ignore_index=True)
final_df.to_csv()

KeyboardInterrupt: 

In [73]:
pages = scribe_htrc['page'].unique()
pages = pages.tolist()
pages
for i, page in enumerate(pages):
    selected_rows = scribe_htrc.loc[scribe_htrc['page'] == page].copy()
    print(page, len(selected_rows.index))

1 7
7 25
8 125
9 52
10 159
11 295
12 352
13 49
15 221
16 194
18 184
19 273
20 95
21 95
22 161
23 159
24 191
25 188
26 273
27 289
28 400
29 420
30 152
31 218
32 178
33 417
34 141
35 370
36 367
37 359
38 290
39 263
40 344
41 332
42 315
43 255
44 189
45 381
46 286
47 323
48 428
49 407
50 476
51 490
52 453
53 444
54 480
55 453
56 502
57 471
58 505
59 501
60 484
61 423
62 472
63 38
64 37
65 51
66 108
67 174
68 298
69 379
70 194
71 273
72 405
73 293
74 345
75 138
76 353
77 141
78 61
80 91
82 229
84 129
85 133
86 7
87 88
88 174
89 162
90 126
91 108
92 189
93 232
94 147
95 277
97 148
98 322
99 255
100 52
102 283
103 316
104 180
105 438
106 202
107 243
108 379
109 184
110 44
111 36
113 10
114 169
115 57
116 189
117 246
118 362
119 152
120 300
121 383
122 166
123 330
124 45
125 102
126 138
127 129
128 313
129 323
130 173
131 289
132 459
133 417
134 59
135 446
136 441
137 420
138 257
139 280
140 323
141 397
142 344
143 284
144 250
145 433
146 383
147 352
148 199
150 52
151 393
152 460
153 485
154

In [67]:
pages = scribe_htrc['page'].unique()
pages = pages.tolist()
pages
for i, page in enumerate(pages):
    selected_rows = scribe_htrc.loc[scribe_htrc['page'] == page].copy()
    print(page, len(selected_rows.index))

2 3
7 196
8 6
9 186
11 173
13 130
15 131
17 136
18 8
19 167
21 198
23 176
24 8
25 190
27 164
29 201
30 15
31 202
32 2
33 131
35 192
36 6
37 166
38 172
39 140
41 106
42 1
43 145
45 123
47 109
48 4
49 132
50 6
51 127
52 1
53 112
55 147
57 141
59 144
61 156
63 125
64 9
65 158
67 140
68 1
69 151
70 1
71 123
73 132
74 9
75 171
76 6
77 125
78 48
79 160
81 157
82 52
83 157
85 133
87 186
89 15
91 63
92 8
93 47
94 4
95 183
96 2
97 118
99 168
101 146
103 96
105 122
107 156
108 1
109 87
111 112
113 133
114 1
115 172
117 139
119 180
121 202
123 189
125 158
127 142
129 150
131 165
133 187
134 1
135 206
137 198
139 156
141 204
143 100
145 105
147 131
149 181
151 172
152 4
153 153
155 169
157 190
159 141
161 110
163 129
164 1
165 121
166 1
167 120
169 113
171 83
173 134
174 8
175 130
177 149
179 118
180 5
181 134
183 134
184 9
185 148
187 170
189 214
190 1
192 8
193 236
195 146
196 8
197 200
198 3
199 85
201 176
203 232
205 104
206 12
207 140
208 2
209 163
211 114
212 3
213 156
215 130
216 4
217 114


1638 4
1639 16
1640 16
1641 58
1643 229
1645 209
1646 12
1647 231
1648 2
1649 207
1650 4
1651 68
1653 131
1655 149
1657 82
1658 1
1659 155
1660 2
1661 145
1663 161
1664 3
1665 119
1666 17
1667 130
1668 29
1669 127
1670 2
1671 163
1673 224
1675 148
1676 2
1677 119
1678 9
1679 119
1680 38
1681 120
1682 16
1683 125
1684 5
1685 84
1686 8
1687 60
1688 11
1696 1
1698 48
1704 35
1710 41
1712 1
1714 2
1716 200
1718 44
1720 4
1722 55
1726 182
1734 54
1736 34
1738 5
1742 1
1744 10
1750 23
1758 2
1760 1
1766 12
1772 1
1774 1
1778 34
1782 1
1784 1
1786 5
1804 7
1806 13
1808 10
1814 29
1816 19
1820 35
1827 5
1828 2
1836 4
1838 12
1840 17
1844 6
1846 1
1848 7
1854 16
1860 7
1862 23
1864 2
1868 73
1876 4
1884 26
1886 33
1888 20
1892 3
1898 25
1900 1
1902 72
1904 35
1907 96
1908 20
1914 11
1916 1
1920 31
1924 2
1928 1
1944 1
1950 23
1962 1


In [ ]:
# Calculate jaccard ratio. Takes list of tokens
jac = 1 - distance.jaccard(order_list, unorder_list)
print('Jaccard', jac)
ocr_values.append(jac)

# use gensim's similarity matrix and lsi to calculate cosine
all_tokens = [order_list, unorder_list]
dictionary = Dictionary(all_tokens)
corpus = [dictionary.doc2bow(text) for text in all_tokens]
lsi = LsiModel(corpus, id2word=dictionary, num_topics=2)
sim = MatrixSimilarity(lsi[corpus])
lsi_cos = [ t[1][1] for t in list(enumerate(sim))]
lsi_cos = lsi_cos[0]
print('LSI', lsi_cos)
ocr_values.append(lsi_cos)